# ROBOFISH

### Imports

In [ ]:
import FISH2_functions

### Find machine address
Use the below function to find the USB address and serial number of the connected devices, one device at a time.  
If you know the identifier of the FTDI chip of your device use it as input.  
Otherwise follow the instructions of the function and unplug and plug your device to find the address.  
  
When the search is successful add the serial number (preferred) or the USB COM port (alternative) to the data file.  
Open the data file with the `ROBOFISH_user_program.py` and add the info to the `ROBOFISH_System_datafile.yaml`.

In [ ]:
FISH2_functions.find_address(identifier=None)

# Initiate system

In [ ]:
import FISH2_functions

#System specific paths to database that is used to keep track of the experiment data. 
#database is automatically generated by the "FISH2_user_program.py" and prints the path after runing, copy the path to this location 
db_path = 'FISH_database\FISH_System2_db.sqlite'

#System specific path to start_imaging_file
start_imaging_file_path = "C:\\Users\\BL\\Desktop\\ROBOFISH_next\\FISH_database\\start_imaging_file.txt"

#System specific path to where the microscope saves the images. The info files will be put here.
#Use double slashes "\\" but do not put the trailing slashes! Example "C:\\Folder\\subfolder"
imaging_output_folder = "C:\\Users\\BL\\Desktop\\ROBOFISH_next\\test"


F2 = FISH2_functions.FISH2(db_path, imaging_output_folder, start_imaging_file_path, system_name='ROBOFISH2')

# Log manual protocol

In [ ]:
prot = """MANUAL PROTOCOL:
Sample preparation performed on 2020-05-19.
Slide prepared on 2020-05-19.
Slide stored at -80C

Label the nucei with 5mg/ml Hoechst in SSC 2X for 5 minutes.
Wash 5 times with SSC 2X.
Place coverslip with parafilm gasket on top of tissue.
Select imaging area and perform reference imaging with 40X objective.

Remove Coverslip, gasket and SSC2X.
Permeabilize the tissue 5 minutes with 0.1% (v/v) Triton X-100, 10mM DTT in 1X TBE.
Wash 5 times with 1X TBE
perform electrophoresis 20 minutes at 1.5V (15V/cm) in 1X TBE, 1M Urea and 10mM DTT.
During next digestion pipet up and down to disloge the tissue.
Digest 3 times 10 minutes with 12.5ul/ml ProtK NEB in digestion buffer at 30C.
Wash 3 times 5 minutes with 5% SDS in 2X SSC at 30C.
Wash 5 times with 2X SSC.
Fix 10 minutes with 4% PFA in 1X PBS.
Wash 3 times with PBS buffer.
Wash 3 times with 2X SSC.

Dry 2ul of Mouse oPool V2 647 with speed vac.
Resuspend in 20ul of 30% formamide hybridization mix, 1nM/probe.
30% Hybridication mix contains:
    2X SSC, 
    0.1 g/ml Dextran sulfate, 
    1mg/ml E. Coli tRNA, 
    2mM Ribonucleoside Vanadyl Complex, 
    200ug/ml Bovine Serum Albumin
    30% (v/v) formamide

Place on tissue and cover with Grace Biolabs Hybrislip.
Incubate at 38.5 C in humidified petridish (paper towel wetted with 2X SSC, covered by parafilm).
Start: 19:30

Stop: 2020-05-21 at 10:00
Mount in Optical Biosystems flow cell and connect to ROBOFISH system

Readout mix in 30% Formamide hybridization mix:
    50mM Fluorescent readout probes.
    1uM Strand displacement probes.
(Strand displacer binds on the readout.)

Wash buffer after encoding probes
    2X SSC,
    30% (v/v) formamide.
    
Wash buffer after readout probes
    2X SSC,
    20% (v/v) formamide.

Imaging buffer:
    2X SSC,
    2mM Trolox (6-hydroxy-2,5,7,8-tetramethylchroman-2-carboxylic acid)
    5mM 3,4-Dihydroxybenzoic acid
    20nM Protocatechuate 3,4-Dioxygenase from Pseudomonas sp.

"""    
F2.L.logger.info(prot)
F2.L.logger.info('____')

# Individual functions

In [ ]:
def EEL_washBufferWash(chamber, volume, cycles, incubation_time=0, buffer='WB'):
    """
    Stringency wash after hybridization. Including SSC2X wash afterwards.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `volume`(int): Volume of WB to use per wash cycle.
    `cycles`(int): Number of cycle to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    
    """
    F2.L.logger.info('Stingency wash {} start'.format(chamber))
    #Wash with WB
    F2.extractDispenseBuffer(buffer, volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer(buffer, volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)
    #Wash out WB
    for i in range(4):
        F2.extractDispenseRunningBuffer(1000, chamber)
    F2.L.logger.info('Stringency wash {}. Washed {} times with {}ul WB for {} minutes'.format(chamber, cycles, volume, incubation_time))
    
def EEL_fix(chamber):
    """
    Perform PFA fixation manually.
    Input:
    `chamber`(str): Chamber to fix. Like: 'Chamber1'.
    
    """
    #Manual Fixation
    long_message='Manually fix 10 minutes with 4% PFA in 1X PBS. Afterwards flush with SSC2X'
    F2.push(short_message='Fixation chamber{}'.format(chamber), long_message=long_message)
    print(long_message)
    #Set temperature
    F2.setTemp(F2.Parameters['Imaging_temperature'], chamber)
    F2.waitTemp(F2.Parameters['Imaging_temperature'], chamber, error=3, sd=0.01, verbose=False)
    input('Perform Fixation. Press enter when done with fixation...')
    F2.L.logger.info('Manually fixed 10 minutes with 4% PFA in 1X PBS.')
    F2.L.logger.info('Manually flushed with PBS and SSC2X.')
    #Set temperature
    F2.setTemp(F2.Parameters['Staining_temperature'], chamber)
    F2.extractDispenseRunningBuffer(1000, chamber)
    F2.waitTemp(F2.Parameters['Staining_temperature'], chamber, error=3, sd=0.01, verbose=False)
    
def EEL_imagingBuffer(chamber, volume, imaging_temp):
    """
    Inject imaging buffer into the chamber and set chamber to imaging temperature.
    Input:
    `chamber`(str): Chamber to inject. Like: 'Chamber1'.
    `volume`(int): Volume of imaging buffer to inject.
    `imaging_temp`(float): Temperature to perform the imaging at.
    
    """
    F2.L.logger.info('Imaging buffer injecting into {}'.format(chamber))
    F2.setTemp(imaging_temp, chamber)
    F2.extractDispenseBuffer('IB', volume, chamber, padding=True, same_buffer_padding=False)
    F2.waitTemp(imaging_temp, chamber, error=3, sd=0.01, verbose=False)
    F2.L.logger.info('Imaging buffer. Injected {}ul of imaging buffer into {}, temperature set to {}C'.format(volume, chamber, imaging_temp))

def EEL_stripping(chamber, volume, cycles, stripping_temp, incubation_time=0, wash_cycles= 5):
    """
    Stripping off hybridized probes. Including SSC2X wash before (to wash out IB)
    and after.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `volume`(int): Volume of SB to use per wash cycle.
    `cycles`(int): Number of cycles to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    `stripping_temp`(int): Temperature used for hybridization/washes in Celcius.
    `wash_cycles`(int): Number of 1000ul washes after the striping. Default=5
    
    """
    F2.L.logger.info('Stripping {} start'.format(chamber))
    #Wash out imaging buffer
    for i in range(4):
        F2.extractDispenseRunningBuffer(1000, chamber)
    F2.extractDispenseBuffer('SB', volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer('SB', volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)   
    #Wash out SB
    for i in range(wash_cycles):
        F2.extractDispenseRunningBuffer(1000, chamber)
    F2.L.logger.info('Stripping. Washed {}, {} times with {}ul SB for {} minutes'.format(chamber, cycles, volume, incubation_time))
    
def EEL_hybmix(chamber, cycle, hybridization_temp, incubation_time, indirect=None,):
    """
    Hybridization.
    Input:
    `chamber`(str): Chamber to hybridize. Like: 'Chamber1'.
    `cycle`(int): Current staining cycle.
    `hybridization_temp`(float): Hybridization temperature.
    `incubation_time`(int): Hybridization time in HOURS.
    `indirect`(str): Amplifier code. 
    
    """
    F2.L.logger.info('Hybridization {}, cycle {} start dispense'.format(chamber, cycle))
    F2.extractDispenseHybmix(chamber, cycle, slow_speed=27, prehyb=False, steps=0, h=incubation_time, indirect=indirect)
    F2.L.logger.info('Hybridization {}, cycle {} finished'.format(chamber, cycle))

# Combined functions

In [ ]:
def EEL_readout(chamber, cycle):
    """
    Perform the first round of the EEL detection
    
    """
    #Set temperature
    F2.setTemp(F2.Parameters['Staining_temperature'], chamber)
    F2.waitTemp(F2.Parameters['Staining_temperature'], chamber, error=3, sd=0.01, verbose=False)
    
    F2.extractDispenseRunningBuffer(1000, chamber)
    #Washing
    EEL_washBufferWash(chamber, 600, 4, 15, buffer='SB') 
    F2.L.logger.info('Replaced 30% formamide WB which was in the SB buffer line, to the TCEP stripping buffer')
     
    #PFA fization
    EEL_fix(chamber)

    #Hybridization
    hyb_time_code = 'Hyb_time_{}_A'.format(chamber[-1])
    hyb_time = F2.Parameters[hyb_time_code]
    EEL_hybmix(chamber, cycle, F2.Parameters['Staining_temperature'], hyb_time)
    #Washing
    
    EEL_washBufferWash(chamber, 600, 3, 3)
    
    #Imaging
    EEL_imagingBuffer(chamber, 600, F2.Parameters['Imaging_temperature'])
    F2.push(short_message='{} Ready for Imaging round: {}'.format(chamber, cycle))
    
    
    
def EEL_readout_repeat(chamber, cycle):
    """
    Perform the repeat round of the EEL detection
    
    """
    #stripping
    #EEL_stripping(chamber, 600, 2, F2.Parameters['Stripping_temperature'], 10, wash_cycles=10)
    #if cycle == 2:
    #    F2.push(short_message='{} Performed first stripping {}'.format(chamber, cycle))
    #    input('Press enter to continue...')
    #Wash out IB if stripping is not performed seperately
    for i in range(4):
        F2.extractDispenseRunningBuffer(1000, chamber)
        
    #Set temperature
    F2.setTemp(F2.Parameters['Staining_temperature'], chamber)
    F2.waitTemp(F2.Parameters['Staining_temperature'], chamber, error=3, sd=0.01, verbose=False)
    
    #Hybridization
    hyb_time_code = 'Hyb_time_{}_A'.format(chamber[-1])
    hyb_time = F2.Parameters[hyb_time_code]
    EEL_hybmix(chamber, cycle, F2.Parameters['Staining_temperature'], hyb_time)
    
    #Washing
    EEL_washBufferWash(chamber, 600, 3, 3) 
    
    #Imaging
    EEL_imagingBuffer(chamber, 600, F2.Parameters['Imaging_temperature'])
    F2.push(short_message='{} Ready for Imaging round: {}'.format(chamber, cycle))

# Scheduler

In [ ]:
F2.scheduler(EEL_readout, EEL_readout_repeat)